# Виконання

## Завдання перше

### За допомогою ChatGPT було згенеровано датасет та за допомогою Brat протеговано сутності. Зчитаємо файл.

In [23]:
import ast
with open('prepared_train_data.txt', 'r') as file:
    trainset = ast.literal_eval(''.join(file.readlines()))
trainset

[('The Grammy Awards ceremony, hosted by the National Academy of Recording Arts and Sciences, showcased exceptional musical talent from around the world.',
  {'entities': [(4, 17, 'WORK_OF_ART'), (38, 89, 'ORG')]}),
 ("The Vienna Philharmonic Orchestra, known for its rich history and exquisite performances, captivated audiences with their rendition of Beethoven's Symphony No.",
  {'entities': [(0, 33, 'ORG'), (135, 144, 'PERSON')]}),
 ('9.', {'entities': [(0, 1, 'CARDINAL')]}),
 ('The Rolling Stones, a legendary rock band formed in London, England, embarked on a world tour to celebrate their 50th anniversary.',
  {'entities': [(0, 18, 'ORG'),
    (52, 58, 'GPE'),
    (60, 67, 'GPE'),
    (113, 117, 'ORDINAL')]}),
 ('The Metropolitan Opera, located in New York City, is renowned for its grand opera productions and attracts opera enthusiasts from all over the globe.',
  {'entities': [(0, 22, 'ORG'), (35, 48, 'GPE')]}),
 ("The National Symphony Orchestra, based in Washington, D.C., collabo

*Навчальний набір даних*

### Навчимо модель, вимкнемо всі компоненти статистичного конвеєра, включаючи тег частини мови і аналізатор залежностей.

In [24]:
import random
import spacy
from spacy.training import Example

nlp = spacy.blank("en")
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

epochs = 25

with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.create_optimizer()
    for i in range(epochs):
        random.shuffle(trainset)
        for text, annotation in trainset:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            nlp.update([example], sgd=optimizer)

*Навчання моделі*

### Продемонструємо роботу.

In [26]:
sentences = [
    'Home is big',
    'The American Society of Music Arrangers and Composers (ASMAC) honored legendary film composer John Williams with a Lifetime Achievement Award for his contributions to the industry.'
]
docs = [nlp(sent) for sent in sentences]
for doc in docs:
    print(doc.ents)

()
(The American Society of Music Arrangers and Composers (ASMAC, John Williams, Lifetime Achievement Award)
